In [1]:
"""
import pandas as pd
df = pd.read_csv("./qingbao.csv",  on_bad_lines='skip')
res = ""
for i in df['origin']:
    res += i + "\n"
with open("./res.txt", "w", encoding='utf-8') as f:
    f.write(res)
"""
import pandas as pd
from GetRelationShips import GetRelationShips
from GetEventType import GetEventType
from GetEventDetail import GetEventDetail
from load_model import chatglm
from extractJson import extractJson, JsonValidate
import json
import tqdm

/root/anaconda3/envs/ChatGLM2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("./qingbao.csv",  on_bad_lines='skip')
res = []
for i in df['origin']:
    res.append(i)

In [3]:
model = chatglm()

Loading checkpoint shards: 100%|██████████| 7/7 [00:12<00:00,  1.82s/it]


In [4]:
cnt = 0
ana_res = []
output = []
for i in tqdm.tqdm(res):
    cnt += 1
    # print(f"progress:{cnt}/{len(res)}")
    # print(f"content:{i}\n\n")
    relation = GetRelationShips(i, model)
    # print(f"relationships={relation}")
    relationJson = extractJson(relation)
    relationJson = relationJson if JsonValidate(relationJson) else 'Json Invalidate'
    # print(f"Json inside={relationJson}")
    if not JsonValidate(relationJson):
        # ana_res.append({"事件描述": i, "事件关系": "Json Invalidate"})
        print("INFO：Json Invalidate")
        continue
    
    output.append(relationJson)
    with open("./relationJson.json", "w", encoding='utf-8') as f:
        json.dump(output, f)
    
    try:
        pos = 0
        relationJson = json.loads(relationJson)
        # print(f"relationJson={relationJson}")
        for item in relationJson["因果关系"]:
            reason = item.split("->")[0]
            result = item.split("->")[1]
            reason_type = GetEventType(reason, model)
            result_type = GetEventType(result, model)
            relationJson["因果关系"][pos] = {
                "原因": {
                    "事件类型": reason_type,
                    "事件细节": GetEventDetail(reason, reason_type, model),
                    "事件描述": reason
                },
                "结果": {
                    "事件类型": result_type,
                    "事件细节": GetEventDetail(result, result_type, model),
                    "事件描述": result
                }
            }
            pos += 1

        pos = 0
        for item in relationJson["顺承关系"]:
            reason = item.split("->")[0]
            result = item.split("->")[1]
            reason_type = GetEventType(reason, model)
            result_type = GetEventType(result, model)
            relationJson["顺承关系"][pos] = {
                "前一事件": {
                    "事件类型": reason_type,
                    "事件细节": GetEventDetail(reason, reason_type, model),
                    "事件描述": reason
                },
                "后一事件": {
                    "事件类型": result_type,
                    "事件细节": GetEventDetail(result, result_type, model),
                    "事件描述": result
                }
            }
            pos += 1
    except Exception as e:
        # relationJson = str(e)
        print(f"ERROR:{e}")
        continue
    
    ana_res.append({"事件描述": i, "事件分析结果": relationJson})
    # if cnt >= 100:
        # break
    with open("./analysis_result.json", 'w', encoding='utf-8') as f:
        json.dump(ana_res, f, indent=4)

  0%|          | 3/1415 [01:32<12:01:53, 30.68s/it]

ERROR:Invalid \escape: line 6 column 23 (char 327)


  0%|          | 4/1415 [01:37<7:56:40, 20.27s/it] 

ERROR:Expecting property name enclosed in double quotes: line 2 column 1 (char 2)


  0%|          | 5/1415 [02:13<10:12:14, 26.05s/it]

...

100%|█████████▉| 1413/1415 [9:38:53<00:42, 21.48s/it]

ERROR:Expecting property name enclosed in double quotes: line 2 column 1 (char 2)


100%|█████████▉| 1414/1415 [9:39:02<00:17, 17.69s/it]

ERROR:Expecting property name enclosed in double quotes: line 2 column 1 (char 2)


100%|██████████| 1415/1415 [9:39:24<00:00, 24.57s/it]


查看事件图谱结果

In [6]:
import pprint
with open("./analysis_result.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
pprint.pprint(data)

[{'事件分析结果': {'因果关系': [{'原因': {'事件描述': '研究人员发现透明部落利用外贸主题的链接进行攻击活动样本',
                              '事件类型': '威胁事件',
                              '事件细节': "{'攻击对象': '透明部落', '攻击者': '未知', '攻击手段': "
                                      "'利用外贸主题的链接进行攻击', '攻击工具': ''}"},
                       '结果': {'事件描述': '透明部落使用一个持久化组件用于驻留',
                              '事件类型': '脆弱性事件',
                              '事件细节': '问题1：\n'
                                      '\n'
                                      '文字：透明部落使用一个持久化组件用于驻留。\n'
                                      '\n'
                                      '事件类型：脆弱性事件\n'
                                      '\n'
                                      "输出：{'事件类型': '脆弱性事件', '具体事件': "
                                      "'透明部落使用一个持久化组件用于驻留'}"}}],
             '顺承关系': [{'前一事件': {'事件描述': '研究人员发现透明部落利用外贸主题的链接进行攻击活动样本',
                                '事件类型': '攻击事件',
                                '事件细节': "{'攻击对象': '透明部落', '攻击者': '未知', '攻击手段': "
 

  '事件描述': 'Evil Corp(EC"，又名"Indrik '
          'Spider")，是一个源自俄罗斯的犯罪集团，自2007年以来一直很活跃。2019年12月，美国财政部外国资产管制处(OFAC)对EC发出制裁，因为其利用Dridex造成了超过1亿美元的经济损失。OFAC起诉后，Evil '
          'Corp组织的TTP发生了变化，从2020年开始，频繁改变有效载荷特征，使用不同的开发工具和初始访问方法。从Dridex转向SocGholish框架，以达到混淆归属的目的，并减少使用Dridex和Bitpaymer。在此期间，此组织不再利用PowerShell '
          'Empire，而是更多地依赖Cobalt '
          'Strike来获得最初的立足点并进行横向渗透。2020年5月，一个新的勒索软件WastedLocker出现。WastedLocker采用了混淆代码技术，并且功能与BitPaymer和Dridex的部分功能相似。2020年12月，一个名为Hades的新勒索软件变体首次出现并被公开报道。Hades是WastedLocker的一个64位编译版本，重要代码和功能与WastedLocker重叠。几个月后，在2021年3月，一个新的变体Phoenix '
          'Locker出现。分析表明，这是Hades的一个重塑版本，几乎没有任何变化。后来，一个名为PayloadBIN的新变体出现，是Phoenix '
          'Locker的延续。BitPaymer、WastedLocker、Hades、Phoenix '
          'Locker、PayloadBIN，这些恶意软件的代码高度重叠并且共享配置，足以证明其来自同一代码库。从历史上看，Evil '
          'Corp运作主要有两种方式，第一种是通过传播Dridex(及其衍生产品)进行僵尸网络操作，开展大规模的垃圾邮件活动。第二种是使用BitPaymer勒索软件进行小批量的勒索。这种长期实行的不同的战术都体现了此组织的复杂程度和高适应性。04针对MS-SQL服务器的攻击分"'},
 {'事件分析结果': {'因果关系': [{'原因': {'事件描述': 'Coinminers样本',
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



下面是对主题抽取的结果做处理(去除意外词重复词和同义词)的代码

In [120]:
import pprint
import json
with open("./EventType.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

import pandas as pd

dic = {"type": []}
for i in data:
    dic['type'].append(i)
pd.DataFrame(dic).to_csv("./EventType.csv")

# pprint.pprint(data)

In [136]:
from word_similarity import WordSimilarity2010
import tqdm
import json
with open("./type_result.json", 'r', encoding='utf-8') as f:
    data1 = json.load(f)
data = []
for i in data1:
    # if i != 'NONE':
    if 1:
        data.append(i)
ws_tool = WordSimilarity2010()
eps = 0.9
def is_same(a, b):
    sim_b = ws_tool.similarity(a, b)
    return sim_b>eps

import ast

def is_list(string):
    try:
        result = ast.literal_eval(string)
        return isinstance(result, list)
    except (SyntaxError, ValueError):
        return False

unique_data = []
for i in tqdm.tqdm(data):
    dup = False
    for k in i:
        if k in 'abcdefghijklmnopqretuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ':
            dup = True
            print(i)
            break
    for j in unique_data:
        if i in j:
            dup = True
            break
    if not dup:
        unique_data.append(i)

unique_data_outof_number_and_badF = []
for i in unique_data:
    try:
        kk = int(i)
    except:
        if not is_list(str(i)):
            unique_data_outof_number_and_badF.append(i)
        else:    
            # print(i)
            pass

def remove_duplicates(lst):
    result = []
    n = len(lst)

    for i in tqdm.tqdm(range(n)):
        is_duplicate = False
        for j in range(i + 1, n):
            # print(lst[i], lst[j])
            if is_same(lst[i], lst[j]):
                is_duplicate = True
                # print(lst[i], lst[j])
                break
        if not is_duplicate:
            result.append(lst[i])

    return result

def check_If_duplicates(lst):
    n = len(lst)

    for i in tqdm.tqdm(range(n)):
        is_duplicate = False
        for j in range(i + 1, n):
            # print(lst[i], lst[j])
            if is_same(lst[i], lst[j]):
                return True
    return False

unique_data_outof_final = remove_duplicates(unique_data_outof_number_and_badF)
    

100%|██████████| 241/241 [00:00<00:00, 133336.93it/s]


NONE
NONE
NONE


100%|██████████| 224/224 [00:00<00:00, 4449.16it/s]


In [137]:
len(data), len(unique_data), len(unique_data_outof_number_and_badF), len(unique_data_outof_final), check_If_duplicates(unique_data_outof_final)

100%|██████████| 224/224 [00:00<00:00, 4306.21it/s]


(241, 224, 224, 224, False)

In [138]:
pd.Series(unique_data_outof_final).to_csv("./Event_Type_final.csv")

In [139]:
pd.Series(unique_data_outof_final).head(20)

0       窃取
1       渗透
2       打击
3     利用木马
4       销毁
5       攻击
6       复制
7       发现
8       感染
9       建立
10      建设
11      混淆
12      学习
13      自查
14      分析
15      状态
16      装载
17      信息
18     重定向
19      制造
dtype: object

查看type_result.json文件

In [ ]:
import pprint
import json
with open("./type_result.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
pprint.pprint(data)